<a href="https://colab.research.google.com/github/chogerlate/Hackathon/blob/main/DataScienceHackathonEasyModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook cotains all the code I use in Hackathon-Online-Data Science
hackathon. Honestly, this is not even cross the base-line but I stil wanna keep it as a memento. The approach that I used in the hackathon was quite stright forward and easy, like It took me 5 mins to do data pre-processing at most. So, it's obviously the reason of low score. but generally speaking, I just want to test my hypothesis that "just push the processed datset into Autogluon an boom.. got a high score. " And well, I could say that it partly true but still true. I would say other than waiting for the training of Autogluon to finish, it pretty much nothing else to mention

![picture](https://drive.google.com/uc?export=view&id=1AbbTe2jTZHOHqw9YWgvnkv7DWUwkPqhO)

# Set up

Load dataset

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -q kaggle

In [ ]:

# connect colab to kaggle through a saved token that store in my Gdrive
# !rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/colab/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                            title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
alphiree/cardiovascular-diseases-risk-prediction-dataset       Cardiovascular Diseases Risk Prediction Dataset    5MB  2023-07-03 12:12:19           2741        118  1.0              
nelgiriyewithana/countries-of-the-world-2023                   Global Country Information Dataset 2023           23KB  2023-07-08 20:37:33           1689         67  1.0              
arnavsmayan/netflix-userbase-dataset                           Netflix Userbase Dataset                          25KB  2023-07-04 07:38:41           2810         67  1.0              
iamsouravbanerjee/data-science-salaries-2023                   Latest Data Scien

In [ ]:
!kaggle competitions download -c hackathon-online-2023-data-science

 52% 1.00M/1.93M [00:00<00:00, 2.04MB/s]
100% 1.93M/1.93M [00:00<00:00, 3.40MB/s]


In [ ]:
!mkdir kaggleData
!unzip /content/hackathon-online-2023-data-science.zip -d /content/kaggleData

Archive:  /content/hackathon-online-2023-data-science.zip
  inflating: /content/kaggleData/Data_Description.csv  
  inflating: /content/kaggleData/Test_data.csv  
  inflating: /content/kaggleData/Train_data.csv  
  inflating: /content/kaggleData/occupation.csv  
  inflating: /content/kaggleData/sample_submission.csv  
  inflating: /content/kaggleData/train_target.csv  
  inflating: /content/kaggleData/type.csv  


# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv("/content/kaggleData/Train_data.csv")
test_ds = pd.read_csv("/content/kaggleData/Test_data.csv")
train_target = pd.read_csv("/content/kaggleData/train_target.csv")

<ipython-input-7-2e402686d33a>:2: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  test_ds = pd.read_csv("/content/kaggleData/Test_data.csv")


# Preprocessing

One hot encode

In [ ]:
categorical_columns = train_data.select_dtypes(include=['object']).columns
categorical_columns

Index(['SubDistrict', 'District', 'Province', 'Q1_Occ', 'Q1_OccCode',
       'Q1_OccInd', 'Q1_IndBD_1', 'Q1_IndBD_2', 'Q1_IndBD_3', 'Q2.1_Occ',
       'Q2.1_OccCode', 'Q2.1_OccInd', 'Q2.2_Occ', 'Q2.2_OccCode',
       'Q2.2_OccInd', 'Q3.3Reason', 'Q4_7Other', 'Q5_9Other', 'Q9_9Other',
       'Q10.1_1', 'Q10.1_2', 'Q10.1_3', 'Q10.1_4', 'Q10.1_5', 'Q10.2_1',
       'Q10.2_2', 'Q10.2_3', 'Q10.2_4', 'Q10.2_5', 'LiveOther', 'AssetOther',
       'OtherAppSpec'],
      dtype='object')

In [ ]:
categorical_columns = train_data.select_dtypes(include=['object']).columns
# One-hot encode categorical columns
from sklearn.preprocessing import LabelEncoder

# Label encode categorical columns
label_encoder = LabelEncoder()
for column in categorical_columns:
    train_data[column] = train_data[column].astype(str)
    train_data[column] = label_encoder.fit_transform(train_data[column])
    train_data[column] = train_data[column].astype(float)

train_data = train_data.replace(888, 0)
train_data = train_data.replace(999,0)

In [ ]:
train_ds = pd.merge(train_data, train_target, on='id', how='inner')

In [ ]:
train_ds

,id,Area,SubDistrict,District,Province,Region,Q1_OccStatus,Q1_Occ,Q1_OccCode,Q1_OccInd,...,Moblie,Facebook,Instragram,Tiktok,Youtube,Twitter,OtherApp,OtherAppSpec,NoSocial,Income
0,1,2,37.0,108.0,30.0,5,3,9.0,38.0,10.0,...,0,0,0,0,0,0,0,0.0,0,1
1,2,3,65.0,3.0,24.0,4,1,151.0,14.0,5.0,...,0,0,0,0,0,0,0,0.0,0,2
2,3,2,0.0,116.0,28.0,2,1,543.0,17.0,5.0,...,0,0,0,0,0,0,0,0.0,0,3
3,4,3,141.0,164.0,7.0,2,1,1813.0,8.0,3.0,...,1,0,0,0,0,0,0,0.0,0,1
4,5,1,136.0,67.0,1.0,1,3,2270.0,34.0,10.0,...,1,0,0,0,0,0,0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19342,19345,3,45.0,13.0,10.0,3,1,361.0,17.0,5.0,...,0,0,0,0,0,0,0,0.0,0,4
19343,19346,2,138.0,147.0,4.0,2,3,1935.0,9.0,3.0,...,0,0,0,0,0,0,0,0.0,0,1
19344,19347,2,237.0,136.0,15.0,3,1,874.0,17.0,5.0,...,0,0,0,0,0,0,0,0.0,0,4
19345,19348,2,250.0,93.0,8.0,4,1,1154.0,14.0,5.0,...,1,0,0,0,0,0,0,0.0,0,2


Get correlated columns that greater than the threshold which is 0.3

In [ ]:
# Step 1: Calculate the correlation between 'Income' and other columns
correlation_with_income = train_ds.corr()['Income']

# Step 2: Set the correlation threshold (e.g., 0.5 for high correlation)
correlation_threshold = 0.3

# Step 3: Keep only the columns with high correlation with 'Income'
high_corr_columns = correlation_with_income[abs(correlation_with_income) >= correlation_threshold].index.tolist()
train_ds_high_corr = train_ds[high_corr_columns]

In [ ]:
train_ds_high_corr

,Q1_OccStatus,Q1_IndBD_3,Q1_OCC_B,Q1_OCC_C,Q1_Occ_Have,Q5_TotalA,Q5_TotalB,Q7_1Day,Q8_2A,Salary,Income
0,3,0.0,0,1,1,8900.0,8900.0,0,0,4,1
1,1,1.0,1,0,0,6000.0,6000.0,6,4200,6,2
2,1,1.0,1,0,0,0.0,0.0,1,10000,3,3
3,1,0.0,1,0,0,0.0,0.0,7,0,6,1
4,3,0.0,0,1,1,0.0,0.0,6,5400,2,1
...,...,...,...,...,...,...,...,...,...,...,...
19342,1,1.0,1,0,0,7800.0,7800.0,7,0,7,4
19343,3,0.0,0,1,1,7000.0,7000.0,7,7000,3,1
19344,1,1.0,1,0,0,16750.0,16750.0,7,30000,7,4
19345,1,1.0,1,0,0,0.0,0.0,7,0,4,2


remove outliers

In [ ]:
# Function to remove outliers using IQR
def remove_outliers_iqr(df, k=3):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    return df[((df >= lower_bound) & (df <= upper_bound)).all(axis=1)]

# Apply the function to remove outliers in the DataFrame
df_no_outliers = remove_outliers_iqr(train_ds_high_corr)

df_no_outliers

,Q1_OccStatus,Q1_IndBD_3,Q1_OCC_B,Q1_OCC_C,Q1_Occ_Have,Q5_TotalA,Q5_TotalB,Q7_1Day,Q8_2A,Salary,Income
1,1,1.0,1,0,0,6000.0,6000.0,6,4200,6,2
2,1,1.0,1,0,0,0.0,0.0,1,10000,3,3
3,1,0.0,1,0,0,0.0,0.0,7,0,6,1
5,1,1.0,1,0,0,0.0,0.0,7,4000,6,3
6,1,1.0,1,0,0,17200.0,17200.0,7,10000,5,3
...,...,...,...,...,...,...,...,...,...,...,...
19341,1,1.0,1,0,0,0.0,0.0,0,11000,7,3
19342,1,1.0,1,0,0,7800.0,7800.0,7,0,7,4
19344,1,1.0,1,0,0,16750.0,16750.0,7,30000,7,4
19345,1,1.0,1,0,0,0.0,0.0,7,0,4,2


Normalize data

In [ ]:
# Use standard scaler to normalize data to normal distribution
from sklearn.preprocessing import StandardScaler, PowerTransformer, Normalizer
nor = Normalizer()

# Normalize data
to_normalize = df_no_outliers.columns[:-1]
df_no_outliers[to_normalize] = nor.fit_transform(df_no_outliers[to_normalize])

<ipython-input-18-b9d6f9fe0b99>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outliers[to_normalize] = nor.fit_transform(df_no_outliers[to_normalize])


In [ ]:
df_no_outliers

,Q1_OccStatus,Q1_IndBD_3,Q1_OCC_B,Q1_OCC_C,Q1_Occ_Have,Q5_TotalA,Q5_TotalB,Q7_1Day,Q8_2A,Salary,Income
1,0.000106,0.000106,0.000106,0.0,0.0,0.633724,0.633724,0.000634,0.443607,0.000634,2
2,0.000100,0.000100,0.000100,0.0,0.0,0.000000,0.000000,0.000100,1.000000,0.000300,3
3,0.107211,0.000000,0.107211,0.0,0.0,0.000000,0.000000,0.750479,0.000000,0.643268,1
5,0.000250,0.000250,0.000250,0.0,0.0,0.000000,0.000000,0.001750,0.999997,0.001500,3
6,0.000038,0.000038,0.000038,0.0,0.0,0.653997,0.653997,0.000266,0.380231,0.000190,3
...,...,...,...,...,...,...,...,...,...,...,...
19341,0.000091,0.000091,0.000091,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.000636,3
19342,0.000091,0.000091,0.000091,0.0,0.0,0.707106,0.707106,0.000635,0.000000,0.000635,4
19344,0.000026,0.000026,0.000026,0.0,0.0,0.438199,0.438199,0.000183,0.784834,0.000183,4
19345,0.121268,0.121268,0.121268,0.0,0.0,0.000000,0.000000,0.848875,0.000000,0.485071,2


# Model

In [ ]:
# CPU version of pytorch has smaller footprint - see installation instructions in
# pytorch documentation - https://pytorch.org/get-started/locally/

!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label="Income", eval_metric='f1_macro').fit(TabularDataset(df_no_outliers),  presets='best_quality', ag_args_fit={'num_gpus': 1})

No path specified. Models will be saved in: "AutogluonModels/ag-20230715_234011/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230715_234011/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   146.68 GB / 179.07 GB (81.9%)
Train Data Rows:    14036
Train Data Columns: 10
Label Column: Income
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	4 unique label values:  [2, 3, 1, 4]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train D

# Predict

In [ ]:
test_ds

,id,Area,SubDistrict,District,Province,Region,Q1_OccStatus,Q1_Occ,Q1_OccCode,Q1_OccInd,...,Line_Level,Moblie,Facebook,Instragram,Tiktok,Youtube,Twitter,OtherApp,OtherAppSpec,NoSocial
0,1,2,โสนลอย,เทศบาลเมืองบางบัวทอง,นนทบุรี,1,1,อู่ซ่อมรถ,E4,E,...,3,888,888,888,888,888,888,888,888,888
1,2,1,ตลาดบางเขน,หลักสี่,กรุงเทพฯ,1,3,รับจ้างทั่วไป,P10,P,...,2,888,888,888,888,888,888,888,888,888
2,3,3,ดอนแก้ว,แม่ริม,เชียงใหม่,3,1,ร้านตัดผมชาย,P4,P,...,888,1,0,0,0,0,0,0,888,0
3,4,2,ในเมือง,เทศบาลนครอุบลราชธานี,อุบลราชธานี,4,1,ขายปลาสด,D1,D,...,888,1,0,0,0,0,0,0,888,0
4,5,1,จอมพล,จตุจักร,กรุงเทพฯ,1,1,ขายต้นไม้,D4,D,...,4,888,888,888,888,888,888,888,888,888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833,4834,3,ดงมูลเหล็ก,เมืองเพชรบูรณ์,เพชรบูรณ์,3,1,ขายเนื้อสด,D4,D,...,4,888,888,888,888,888,888,888,888,888
4834,4835,2,ตลาดใหญ่,เทศบาลนครภูเก็ต,ภูเก็ต,5,1,ขายโทรศัพท์,D1,D,...,4,888,888,888,888,888,888,888,888,888
4835,4836,2,เทศบาลตำบลสามง่าม,ดอนตูม,นครปฐม,2,1,ขายหมูสด,D4,D,...,3,888,888,888,888,888,888,888,888,888
4836,4837,2,เวียง,เทศบาลนครเชียงราย,เชียงราย,3,1,ขายผลไม้,D4,D,...,3,888,888,888,888,888,888,888,888,888


In [ ]:
categorical_columns = test_ds.select_dtypes(include=['object']).columns

label_encoder = LabelEncoder()

for column in categorical_columns:
    test_ds[column] = test_ds[column].astype(str)
    test_ds[column] = label_encoder.fit_transform(test_ds[column])
    test_ds[column] = test_ds[column].astype(float)

test_ds = test_ds.replace(888, 0)
test_ds = test_ds.replace(999,0)

In [ ]:
# Use standard scaler to normalize data to normal distribution
from sklearn.preprocessing import StandardScaler, PowerTransformer, Normalizer
nor = Normalizer()

# Get high corr
test_ds_high_corr = test_ds[high_corr_columns[:-1]]

# Normalize data
to_normalize = test_ds_high_corr.columns
test_ds_high_corr[to_normalize] = nor.transform(test_ds_high_corr[to_normalize])


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but Normalizer was fitted without feature names
  warnings.warn(
<ipython-input-32-3992f25c4016>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ds_high_corr[to_normalize] = nor.transform(test_ds_high_corr[to_normalize])


In [ ]:
result = predictor.predict(test_ds_high_corr)

In [ ]:
result

0       2
1       1
2       1
3       4
4       4
       ..
4833    4
4834    4
4835    4
4836    4
4837    4
Name: Income, Length: 4838, dtype: int64

# Submission

In [ ]:
sub = pd.read_csv("/content/kaggleData/sample_submission.csv")
sub.head()

,id,Predict
0,1,2.0
1,2,1.0
2,3,1.0
3,4,NaN
4,5,NaN


In [ ]:
sub['Predict'] = result

In [ ]:
sub['Predict'] = sub['Predict'].astype(np.uint8)
sub.head()

,id,Predict
0,1,2
1,2,1
2,3,1
3,4,4
4,5,4


In [ ]:
sub.to_csv("submit.csv",index=False)

In [ ]:
!kaggle competitions submit hackathon-online-2023-data-science -f /content/submit.csv -m "Well submission"

100% 32.0k/32.0k [00:00<00:00, 66.8kB/s]
Successfully submitted to Hackathon-Online-Data Science